# Calculating metrics from pytorch NN output for categorical variables.

In [1]:
import torch
# import our library
import torchmetrics 

In [2]:
metric = torchmetrics.Accuracy()

In [3]:
n_batches = 10
for i in range(n_batches):
    # simulate a classification problem
    preds = torch.randn(10, 5).softmax(dim=-1)
    target = torch.randint(5, (10,))

    # metric on current batch
    acc = metric(preds, target)
    print(f"Accuracy on batch {i}: {acc}")    

# metric on all batches using custom accumulation
acc = metric.compute()
print(f"Accuracy on all data: {acc}")

Accuracy on batch 0: 0.10000000149011612
Accuracy on batch 1: 0.10000000149011612
Accuracy on batch 2: 0.20000000298023224
Accuracy on batch 3: 0.20000000298023224
Accuracy on batch 4: 0.5
Accuracy on batch 5: 0.20000000298023224
Accuracy on batch 6: 0.10000000149011612
Accuracy on batch 7: 0.20000000298023224
Accuracy on batch 8: 0.30000001192092896
Accuracy on batch 9: 0.20000000298023224
Accuracy on all data: 0.20999999344348907


In [4]:
preds

tensor([[0.1605, 0.3604, 0.1251, 0.2884, 0.0656],
        [0.3127, 0.0379, 0.2730, 0.2690, 0.1075],
        [0.3782, 0.1159, 0.2667, 0.2001, 0.0391],
        [0.1932, 0.4260, 0.1140, 0.0272, 0.2396],
        [0.0190, 0.0523, 0.2298, 0.2617, 0.4371],
        [0.4058, 0.1006, 0.1238, 0.1337, 0.2360],
        [0.0565, 0.2157, 0.0974, 0.3100, 0.3204],
        [0.5165, 0.0437, 0.2613, 0.0946, 0.0839],
        [0.2227, 0.3970, 0.1039, 0.0879, 0.1885],
        [0.0611, 0.0175, 0.5535, 0.2605, 0.1075]])

### Input:  
1. predicted values from NN and 
2. actual values/labels.

You will want to be able to see if a predicted value is the same as the actual value. i.e.  
predicted == actual.squeeze (1)). sum () calculates the number of all instances where the predicted value is the same as the actual value.

### Potential output: 
accuracy, precision, recall, TP, TN, FP, FN, IoU, etc.

## Note: double check datatypes.  Use the int() method to convert torch objects to ints.

### Adapted from: 
https://www.tutorialfor.com/blog-282508.htm

https://en.wikipedia.org/wiki/Sensitivity_and_specificity

| Key ||
|:--|:--|
|P |positive instances of a condition |
|N |negative instances of a condition|
|TP |true positive|
|TN |true negative|
|FP |false positive|
|FN |false negative|

|total pop = P+N|Predicted condition positive (PP)|Predicted condition negative (PN)|
|:--|:-:|:-:|
|Actual condition positive (P)|TP (= hit) |FN (Type II error)  Underestimation|
|Actual condition positive (N)|FP (Type I error) Overestimation |TN (= correct rejection)|

In [5]:
predicted = preds.numpy()
actual = target.numpy()

In [15]:
predicted

array([[0.34896025, 0.24110317, 0.06090532, 0.1837537 , 0.16527757],
       [0.04727532, 0.12720998, 0.6532213 , 0.03946552, 0.1328279 ],
       [0.07913141, 0.25618678, 0.16246547, 0.34483114, 0.15738522],
       [0.01374449, 0.23489994, 0.02236762, 0.6424571 , 0.08653083],
       [0.17110842, 0.28075624, 0.16399515, 0.03512565, 0.34901458],
       [0.13203181, 0.3357053 , 0.25312364, 0.16717713, 0.11196211],
       [0.17804138, 0.42529374, 0.15184651, 0.09479353, 0.15002483],
       [0.20513968, 0.06917416, 0.04350902, 0.43064734, 0.25152978],
       [0.28718507, 0.03871686, 0.16038804, 0.07408937, 0.43962067],
       [0.05545131, 0.28975555, 0.13945523, 0.12559803, 0.38973993]],
      dtype=float32)

In [8]:
predicted_classes = torch.argmax(preds, dim=1) == 0
predicted_classes

tensor([False,  True,  True, False, False,  True, False,  True, False, False])

# Accuracy

accuracy = tp+tn/(p+n) = tp+tn/(tp+tn+fp+fn)

Basically, you can find the number of values where predicted == actual, add them up, and divide by the number in the dataset (i.e. training dataset...it needs to be specific to what dataset you are working with).  It is essentially an average.

In [19]:
# total number of correct predictions
train_correct = (predicted == actual.squeeze(1)).sum()

#Used to calculate the accuracy
#train_acc += train_correct.data [0]

#Accuracy:
#train_acc/(len (train_data))

ValueError: cannot select an axis to squeeze out which has size not equal to one

In [ ]:
.squeeze (1)

# At this point, double check datatypes.  Use the int() method to convert torch objects to ints.

In [ ]:
# Pseudocode, assumes 1 = pos, 0 = false for pred == actual

# All 0 variables
# torch.zeros Returns a tensor filled with the scalar value 0, with the shape defined by 
# the variable argument size.
zeros = variable(torch.zeros(lasize).type (torch.longtensor)) 

#All 1 variables
ones = variable (torch.ones (lasize) .type (torch.longtensor)) 

#The total number of original labels is 1, and the prediction is 0
train_correct01 = ((pred == zeros) & (batch_y.squeeze (1) == ons)). sum () 

#The total number of original labels is 0 and the prediction is 1.
train_correct10 = ((pred == ones) & (batch_y.squeeze (1) == zes)). sum () 

# true pos
train_correct11 = ((pred_y == ones) & (batch_y.squeeze (1) == ons)). sum ()

# true neg
train_correct00 = ((pred_y == zeros)&(batch_y.squeeze (1) == zes)). sum ()

# total number of incorrect predictions in the training set:

# false neg
fn += train_correct01.data [0]

# false pos
fp += train_correct10.data [0]

# true pos
tp += train_correct11.data [0]

# true neg
tn += train_correct00.data [0]

# False positive rate:
# ctw + wtc
(fn + fp)/(len (train_data)) 

# Accuracy and recall
#Accuracy
p = tp/(tp + fp)

#Recall rate
r = tp/(tp + fn)

# Real and false positive rates

#True rate:
tpr = tp/(tp + fn)

#False positive rate:
fpr = fp/(fp + tn)

#IoU
iou = tp / (tp + fp + fn)